In [5]:
!pip install torch


  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision.utils import save_image

# Device Config

In [7]:
device = torch.device('cuda')

# Hyperparameters

In [8]:
latent_dim = 100
image_size = 28
image_channels = 1
batch_size = 64
num_epochs = 100
learning_rate = 0.0002

# Generator

In [19]:
class Generator(nn.Module):
  def __init__(self, latent_dim):
    super(Generator, self).__init__()
    self.model = nn.Sequential(
        nn.Linear(latent_dim, 128),
        nn.LeakyReLU(0.2, inplace=True),
        nn.Linear(128, 256),
        nn.BatchNorm1d(256),
        nn.Linear(256, 512),
        nn.ReLU(True),
        nn.Linear(512, 1024),
        nn.ReLU(True),
        nn.Linear(1024, image_channels * image_size * image_size),
        nn.Tanh()
    )
  def forward(self, z):
    return self.model(z)
    img = img.view(img.size(0), image_channels, image_size, image_size)
    return img

# Discriminator

In [58]:
class Discriminator(nn.Module):
  def __init__(self):
    super(Discriminator, self).__init__()
    self.model = nn.Sequential(
        nn.Linear(784, 512),
        nn.LeakyReLU(0.2, inplace=True),
        nn.Linear(512, 256),
        nn.LeakyReLU(0.2, inplace=True),
        nn.Linear(256, 1),
        nn.Sigmoid()
    )

  def forward(self, x):
    x = x.view(x.size(0), -1)
    x = self.model(x)
    return x

# Initilizing Generator and Discriminator

In [59]:
generator = Generator(latent_dim).to(device)
discriminator = Discriminator().to(device)

# Loading Dataset and Defining Data Loader

In [60]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

dataset = datasets.MNIST(root='data', train=True, transform=transform, download=True)
data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Loss Function and Optimizers

In [61]:
adversarial_loss = nn.BCELoss()

optimizer_G = optim.Adam(generator.parameters(), lr=learning_rate, betas = (0.5, 0.999))
optimizer_D = optim.Adam(discriminator.parameters(), lr=learning_rate, betas = (0.5, 0.999))

# GAN Training

In [62]:
for epoch in range(num_epochs):
  for i, (imgs, _) in enumerate(data_loader):
    #config_input
    real_imgs = imgs.to(device)

    # generate real and fake labels
  valid = torch.ones((imgs.size(0), 1), requires_grad=False).to(device)
  fake = torch.zeros((imgs.size(0),1 ), requires_grad=False).to(device)

  optimizer_D.zero_grad()

  z = torch.randn((imgs.size(0), latent_dim)).to(device)
  gen_imgs = generator(z)

  real_loss = adversarial_loss(discriminator(real_imgs), valid)
  fake_loss = adversarial_loss(discriminator(gen_imgs.detach()), fake)
  d_loss = (real_loss + fake_loss) / 2

  d_loss.backward()
  optimizer_D.step()

  #training Generator

  optimizer_G.zero_grad()

  z = torch.randn((imgs.size(0), latent_dim)).to(device)
  gen_imgs = generator(z)

  g_loss = adversarial_loss(discriminator(gen_imgs), valid)

  g_loss.backward()
  optimizer_G.step()

  if i % 100 == 0:
    print(f"Epoch [{epoch}/{num_epochs}] Batch [{i}/{len(data_loader)}] Loss D: {d_loss.item()} Loss G: {g_loss.item()}")

    if epoch % 10 == 0:
      save_image(gen_imgs.data[:25], f"images/{epoch}.png", nrow=5, normalize=True)

# Generate and Save Imgs

In [65]:
import os
import torch
from torchvision.utils import save_image

# ... your existing code ...

# Create the 'images' directory if it doesn't exist
os.makedirs("images", exist_ok=True)

z = torch.randn(64, latent_dim).to(device)
gen_imgs = generator(z)
save_image(gen_imgs.data, "images/generated.png", nrow=8, normalize=True)